In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as sp

def main_simulation(system,t_sim):
    ''' Finds a proper r value for a known simulation, and then 
    runs a simulation using an unknown time (t_sim) to find the 
    temperature at the later time'''
    system['r'] = find_r(system)
    system['t_end'] = t_sim
    results = run_simulation(system,change_func)
    print("With a heat transfer coeffient of r =", round(system['r'],5),
          ", the final temperature is ", round(system['T_final'],4), 
          "after ", t_sim, " minutes.")
    results.plot(xlabel='time (s)', ylabel='Temperature (degrees C', 
                 title='Temperature of Cooling Object', legend=True, label='Coffee');
    return

def make_system(T_init, volume, r, t_end, T_env,T_goal):
    return dict(T_init=T_init, T_final=T_init, volume=volume,
                  r=r, t_end=t_end, T_env=T_env, T_goal=T_goal,
                  t_0=0,  dt=1)

def make_state(system):
    return pd.Series(dict(temp=system['T_init']), name='State object')

def run_simulation(system, change_func):
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)
    state = make_state(system)
    results = pd.Series(index=t_array,dtype=object)
    results.iloc[0] = state.temp
    
    for i in range(n-1):
        t = t_array[i]
        T = state.temp
        delta = change_func(t, state, system)
        state.temp = state.temp + delta
        results.iloc[i+1] = state.temp
    
    system['T_final'] = results.iloc[-1]
    return results

def change_func(t, state, system):
    r, T_env, dt = system['r'], system['T_env'], system['dt']    
    return -r * (state.temp - T_env) * dt

def error_func(r, system):
    system['r'] = r
    results = run_simulation(system, change_func)
    return system['T_final'] - system['T_goal']

def find_r(system):
    root_obj = sp.root_scalar(error_func, system, bracket=[0, 1.0])
    return root_obj.root


In [ ]:
T_init = 90.0;  volume = 300; r = 0.0115; 
t_end = 30; T_env = 22; T_goal = 70.0
system = make_system(T_init, volume, r, t_end, T_env,T_goal)
t_simulation = 50
results = main_simulation(system, t_simulation)
